In [11]:
import os
import shutil

# Klasörlerin yollarını belirleyin
images_folder = 'C:/Users/edana/Desktop/dataset/images'
labels_folder = 'C:/Users/edana/Desktop/dataset/labels'
classes_file = 'C:/Users/edana/Desktop/dataset/classes.txt'

# Çıktı klasörleri
output_folders = {
    'Bird': {'images': 'Bird/images', 'labels': 'Bird/labels'},
    'UAV': {'images': 'UAV/images', 'labels': 'UAV/labels'},
    'Bird and UAV': {'images': 'Bird_and_UAV/images', 'labels': 'Bird_and_UAV/labels'},
}

# Çıktı klasörlerini oluşturun
for output in output_folders.values():
    os.makedirs(output['images'], exist_ok=True)
    os.makedirs(output['labels'], exist_ok=True)

# Sınıfları yükleyin
with open(classes_file, 'r') as file:
    classes = [line.strip() for line in file.readlines()]

# Resim formatları
image_extensions = ['.jpg', '.jpeg', '.webp']

# Etiket dosyalarını işle
for label_file in os.listdir(labels_folder):
    label_path = os.path.join(labels_folder, label_file)
    
    # İlgili görüntü dosyasını bulmak için uzantıları dene
    image_found = False
    for ext in image_extensions:
        image_path = os.path.join(images_folder, label_file.replace('.txt', ext))
        if os.path.exists(image_path):
            image_found = True
            break
    
    if not image_found:
        print(f"Warning: Image file not found for label: {label_file}")
        continue

    with open(label_path, 'r') as file:
        lines = file.readlines()

    labels = set()
    for line in lines:
        class_id = int(line.split()[0])
        labels.add(classes[class_id])

    # Sınıfa göre dosyaları taşı
    if 'Bird' in labels and 'UAV' in labels:
        category = 'Bird and UAV'
    elif 'Bird' in labels:
        category = 'Bird'
    elif 'UAV' in labels:
        category = 'UAV'
    else:
        continue  # İlgili kategori bulunamadıysa geç

    # Dosyaları ilgili klasöre taşı
    shutil.copy(image_path, output_folders[category]['images'])
    shutil.copy(label_path, output_folders[category]['labels'])

print("İşlem tamamlandı.")

İşlem tamamlandı.


Açıklama:

image_extensions Listesi: '.jpg', '.jpeg', ve '.webp' uzantılarını kontrol eden bir liste ekledik.

Görüntü Dosyası Bulma: Etiket dosyasına karşılık gelen görüntü dosyasını bulmak için farklı uzantılar üzerinde döngü yapıyoruz.

Hata Mesajı: Eğer bir görüntü dosyası bulunamazsa, bu durumu print ile bildiriyoruz.

Bu kod ile .jpg, .jpeg, ve .webp formatındaki görüntü dosyaları kontrol edilerek, uygun klasörlere taşınacaktır. 

In [52]:
import os
import shutil

# Klasörlerin yollarını tanımlayın
source_folder = 'C:/Users/edana/Desktop/drone_dataset'  # drone_dataset klasörünün yolu
labels_folder = os.path.join(source_folder, 'labels')  # Etiket dosyalarının toplanacağı klasör
images_folder = os.path.join(source_folder, 'images')  # Görsellerin toplanacağı klasör

# Çıktı klasörlerini oluştur
os.makedirs(labels_folder, exist_ok=True)
os.makedirs(images_folder, exist_ok=True)

# Resim formatları
image_extensions = ['.jpg', '.jpeg', '.webp']

# drone_dataset klasöründeki tüm dosyaları işle
for file_name in os.listdir(source_folder):
    source_path = os.path.join(source_folder, file_name)

    # Eğer dosya bir txt dosyasıysa labels klasörüne kopyala
    if file_name.endswith('.txt'):
        shutil.copy(source_path, labels_folder)

    # Eğer dosya bir resim dosyasıysa images klasörüne kopyala
    elif any(file_name.endswith(ext) for ext in image_extensions):
        shutil.copy(source_path, images_folder)

print("Dosyalar başarıyla taşındı.")

Dosyalar başarıyla taşındı.


Açıklama:

Klasör Yolları: source_folder değişkeni drone_dataset klasörünün yolunu belirtir. labels_folder ve images_folder ise bu klasörün altında oluşturulacak alt klasörlerin yollarını tanımlar.

Klasör Oluşturma: os.makedirs fonksiyonu, belirtilen yolları oluşturur. exist_ok=True parametresi, klasörler zaten varsa hata vermemesini sağlar.

Dosya İşleme: os.listdir fonksiyonu ile drone_dataset klasöründeki tüm dosyalar alınır. .txt dosyaları ve belirtilen resim formatındaki dosyalar, uygun klasörlere kopyalanır.

Resim Formatları: image_extensions listesi, desteklenen resim formatlarını belirtir.

Bu kodu çalıştırarak drone_dataset klasöründeki dosyaları düzenleyebiliriz.

In [ ]:
#BIRD İÇİN DATA AUGMENTATION 1

In [29]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import imgaug.augmenters as iaa
import shutil

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
augmented_images_folder = os.path.join(dataset_folder, 'augmented_images')
augmented_labels_folder = os.path.join(dataset_folder, 'augmented_labels')

# Çıktı klasörlerini oluşturun
os.makedirs(augmented_images_folder, exist_ok=True)
os.makedirs(augmented_labels_folder, exist_ok=True)

# Veri artırma işlemleri için imgaug kullanarak bir artırma zinciri tanımlayın
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Yatay çevirme
    iaa.Flipud(0.5),  # Dikey çevirme
    iaa.Affine(rotate=(-25, 25)),  # Rastgele döndürme
    iaa.Affine(scale=(0.8, 1.2)),  # Rastgele yakınlaştırma
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Rastgele gürültü ekleme
    iaa.Multiply((0.8, 1.2)),  # Renklendirme (görselin parlaklığını rastgele artırma/azaltma)
    iaa.LinearContrast((0.75, 1.5)),  # Kontrast artırma
    iaa.CropAndPad(percent=(0, 0.1)),  # Görseli rastgele kesme ve padleme
    iaa.GaussianBlur(sigma=(0, 3.0))  # Blurlama
])

def color_jitter(image):
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(np.random.uniform(0.8, 1.2))  # Rastgele renk ayarı
    return image

# Etiket dosyalarını ve görselleri işle
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli aç
        image = Image.open(image_path)
        image_np = np.array(image)

        # imgaug ile artırma
        image_aug_np = augmenter.augment_image(image_np)
        image_aug = Image.fromarray(image_aug_np)

        # Renklendirme işlemi
        image_aug = color_jitter(image_aug)

        # Artırılmış görüntüyü kaydet
        augmented_image_path = os.path.join(augmented_images_folder, file_name)
        image_aug.save(augmented_image_path)

        # Etiket dosyasını kopyala (Bu örnekte etiketlerde değişiklik yapılmaz)
        if os.path.exists(label_path):
            shutil.copy(label_path, augmented_labels_folder)

print("Veri artırma tamamlandı.")

Veri artırma tamamlandı.


Açıklamalar

Yatay ve Dikey Çevirme: Görselleri yatay ve dikey olarak çevirir.

Döndürme ve Yakınlaştırma: Görselleri rastgele döndürür ve yakınlaştırır.

Gürültü Ekleme: Görsellere rastgele gürültü ekler.

Renklendirme (Color Jittering): Görsellerin parlaklığını ve kontrastını değiştirir.

Blurlama (Gaussian Blur): Görsellere rastgele blurlama uygular.

Kontrast Artırma: Görsellerin kontrastını artırır.

Kesme ve Padleme (Crop and Pad): Görselleri rastgele keser ve padler.

Bu kod, görselleri çeşitli dönüşümlerle artırır ve etiket dosyalarını uygun yerlere kopyalar. 
Etiketlerin güncellenmesi gerektiği durumlar için daha karmaşık işlemler gerekebilir. 

Renklendirme (Color Jitter): color_jitter fonksiyonu, PIL.ImageEnhance.Color kullanarak rastgele renk ayarları yapar. Görselin renk doygunluğunu artırır veya azaltır.

Blurlama ve Diğer Augmenter: imgaug kullanılarak görsellere blurlama ve diğer dönüşümler uygulanır.

Rastgele Parametreler: Renk ayarları ve diğer dönüşüm parametreleri rastgele seçilir, böylece her çalıştırmada farklı sonuçlar elde edilir.

In [ ]:
#UAV İÇİN DATA AUGMENTATION 1

In [1]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import imgaug.augmenters as iaa
import shutil

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
augmented_images_folder = os.path.join(dataset_folder, 'augmented_images')
augmented_labels_folder = os.path.join(dataset_folder, 'augmented_labels')

# Çıktı klasörlerini oluşturun
os.makedirs(augmented_images_folder, exist_ok=True)
os.makedirs(augmented_labels_folder, exist_ok=True)

# Veri artırma işlemleri için imgaug kullanarak bir artırma zinciri tanımlayın
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Yatay çevirme
    iaa.Flipud(0.5),  # Dikey çevirme
    iaa.Affine(rotate=(-25, 25)),  # Rastgele döndürme
    iaa.Affine(scale=(0.8, 1.2)),  # Rastgele yakınlaştırma
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Rastgele gürültü ekleme
    iaa.Multiply((0.8, 1.2)),  # Renklendirme (görselin parlaklığını rastgele artırma/azaltma)
    iaa.LinearContrast((0.75, 1.5)),  # Kontrast artırma
    iaa.CropAndPad(percent=(0, 0.1)),  # Görseli rastgele kesme ve padleme
    iaa.GaussianBlur(sigma=(0, 3.0))  # Blurlama
])

def color_jitter(image):
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(np.random.uniform(0.8, 1.2))  # Rastgele renk ayarı
    return image

# Etiket dosyalarını ve görselleri işle
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli aç
        image = Image.open(image_path)
        image_np = np.array(image)

        # imgaug ile artırma
        image_aug_np = augmenter.augment_image(image_np)
        image_aug = Image.fromarray(image_aug_np)

        # Renklendirme işlemi
        image_aug = color_jitter(image_aug)

        # Artırılmış görüntüyü kaydet
        augmented_image_path = os.path.join(augmented_images_folder, file_name)
        image_aug.save(augmented_image_path)

        # Etiket dosyasını kopyala (Bu örnekte etiketlerde değişiklik yapılmaz)
        if os.path.exists(label_path):
            shutil.copy(label_path, augmented_labels_folder)

print("Veri artırma tamamlandı.")

Veri artırma tamamlandı.


In [ ]:
#BIRD AND UAV İÇİN DATA AUGMENTATION 1

In [3]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import imgaug.augmenters as iaa
import shutil

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird_and_UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
augmented_images_folder = os.path.join(dataset_folder, 'augmented_images')
augmented_labels_folder = os.path.join(dataset_folder, 'augmented_labels')

# Çıktı klasörlerini oluşturun
os.makedirs(augmented_images_folder, exist_ok=True)
os.makedirs(augmented_labels_folder, exist_ok=True)

# Veri artırma işlemleri için imgaug kullanarak bir artırma zinciri tanımlayın
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Yatay çevirme
    iaa.Flipud(0.5),  # Dikey çevirme
    iaa.Affine(rotate=(-25, 25)),  # Rastgele döndürme
    iaa.Affine(scale=(0.8, 1.2)),  # Rastgele yakınlaştırma
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Rastgele gürültü ekleme
    iaa.Multiply((0.8, 1.2)),  # Renklendirme (görselin parlaklığını rastgele artırma/azaltma)
    iaa.LinearContrast((0.75, 1.5)),  # Kontrast artırma
    iaa.CropAndPad(percent=(0, 0.1)),  # Görseli rastgele kesme ve padleme
    iaa.GaussianBlur(sigma=(0, 3.0))  # Blurlama
])

def color_jitter(image):
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(np.random.uniform(0.8, 1.2))  # Rastgele renk ayarı
    return image

# Etiket dosyalarını ve görselleri işle
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli aç
        image = Image.open(image_path)
        image_np = np.array(image)

        # imgaug ile artırma
        image_aug_np = augmenter.augment_image(image_np)
        image_aug = Image.fromarray(image_aug_np)

        # Renklendirme işlemi
        image_aug = color_jitter(image_aug)

        # Artırılmış görüntüyü kaydet
        augmented_image_path = os.path.join(augmented_images_folder, file_name)
        image_aug.save(augmented_image_path)

        # Etiket dosyasını kopyala (Bu örnekte etiketlerde değişiklik yapılmaz)
        if os.path.exists(label_path):
            shutil.copy(label_path, augmented_labels_folder)

print("Veri artırma tamamlandı.")

Veri artırma tamamlandı.


In [5]:
#BIRD ICIN RESIZE İŞLEMİ 1 W OPENCV

""" 
OpenCV ile Görüntüleri Yeniden Boyutlandırma ve Etiketleri Güncelleme
OpenCV kullanarak görüntüleri yeniden boyutlandırmak için cv2.resize fonksiyonunu kullanabiliriz. 
Bu işlemde, etiketlerin (bounding box) koordinatlarını da yeni boyutlara uygun olacak şekilde güncelleyeceğiz.
"""

' \nOpenCV ile Görüntüleri Yeniden Boyutlandırma ve Etiketleri Güncelleme\nOpenCV kullanarak görüntüleri yeniden boyutlandırmak için cv2.resize fonksiyonunu kullanabiliriz. \nBu işlemde, etiketlerin (bounding box) koordinatlarını da yeni boyutlara uygun olacak şekilde güncelleyeceğiz.\n'

In [3]:
import os
import cv2

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
resized_images_folder = os.path.join(dataset_folder, 'resized_images')
resized_labels_folder = os.path.join(dataset_folder, 'resized_labels')

# Yeniden boyutlandırma hedef boyutları
new_width = 256
new_height = 256

# Çıktı klasörlerini oluşturun
os.makedirs(resized_images_folder, exist_ok=True)
os.makedirs(resized_labels_folder, exist_ok=True)

# Görüntüleri ve etiketleri işleyin
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        # Görüntü dosyasını oku
        image_path = os.path.join(images_folder, file_name)
        image = cv2.imread(image_path)
        h, w = image.shape[:2]

        # Görüntüyü yeniden boyutlandır
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        resized_image_path = os.path.join(resized_images_folder, file_name)
        cv2.imwrite(resized_image_path, resized_image)

        # Etiket dosyasını oku ve güncelle
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        if os.path.exists(label_path):
            resized_label_path = os.path.join(resized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(label_path, 'r') as f:
                lines = f.readlines()
            with open(resized_label_path, 'w') as f:
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        label, x1, y1, x2, y2 = parts
                        # Koordinatları yeniden boyutlandırılmış resme uygun hale getir
                        x1 = int(float(x1) * new_width / w)
                        y1 = int(float(y1) * new_height / h)
                        x2 = int(float(x2) * new_width / w)
                        y2 = int(float(y2) * new_height / h)
                        f.write(f"{label} {x1} {y1} {x2} {y2}\n")

print("Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.")

Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.


In [ ]:
#UAV ICIN RESIZE İŞLEMİ 1 W OPENCV

In [7]:
import os
import cv2

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
resized_images_folder = os.path.join(dataset_folder, 'resized_images')
resized_labels_folder = os.path.join(dataset_folder, 'resized_labels')

# Yeniden boyutlandırma hedef boyutları
new_width = 256
new_height = 256

# Çıktı klasörlerini oluşturun
os.makedirs(resized_images_folder, exist_ok=True)
os.makedirs(resized_labels_folder, exist_ok=True)

# Görüntüleri ve etiketleri işleyin
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        # Görüntü dosyasını oku
        image_path = os.path.join(images_folder, file_name)
        image = cv2.imread(image_path)
        h, w = image.shape[:2]

        # Görüntüyü yeniden boyutlandır
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        resized_image_path = os.path.join(resized_images_folder, file_name)
        cv2.imwrite(resized_image_path, resized_image)

        # Etiket dosyasını oku ve güncelle
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        if os.path.exists(label_path):
            resized_label_path = os.path.join(resized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(label_path, 'r') as f:
                lines = f.readlines()
            with open(resized_label_path, 'w') as f:
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        label, x1, y1, x2, y2 = parts
                        # Koordinatları yeniden boyutlandırılmış resme uygun hale getir
                        x1 = int(float(x1) * new_width / w)
                        y1 = int(float(y1) * new_height / h)
                        x2 = int(float(x2) * new_width / w)
                        y2 = int(float(y2) * new_height / h)
                        f.write(f"{label} {x1} {y1} {x2} {y2}\n")

print("Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.")

Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.


In [ ]:
#BIRD AND UAV ICIN RESIZE İŞLEMİ 1 W OPENCV

In [9]:
import os
import cv2

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird_and_UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
resized_images_folder = os.path.join(dataset_folder, 'resized_images')
resized_labels_folder = os.path.join(dataset_folder, 'resized_labels')

# Yeniden boyutlandırma hedef boyutları
new_width = 256
new_height = 256

# Çıktı klasörlerini oluşturun
os.makedirs(resized_images_folder, exist_ok=True)
os.makedirs(resized_labels_folder, exist_ok=True)

# Görüntüleri ve etiketleri işleyin
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        # Görüntü dosyasını oku
        image_path = os.path.join(images_folder, file_name)
        image = cv2.imread(image_path)
        h, w = image.shape[:2]

        # Görüntüyü yeniden boyutlandır
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        resized_image_path = os.path.join(resized_images_folder, file_name)
        cv2.imwrite(resized_image_path, resized_image)

        # Etiket dosyasını oku ve güncelle
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        if os.path.exists(label_path):
            resized_label_path = os.path.join(resized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(label_path, 'r') as f:
                lines = f.readlines()
            with open(resized_label_path, 'w') as f:
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        label, x1, y1, x2, y2 = parts
                        # Koordinatları yeniden boyutlandırılmış resme uygun hale getir
                        x1 = int(float(x1) * new_width / w)
                        y1 = int(float(y1) * new_height / h)
                        x2 = int(float(x2) * new_width / w)
                        y2 = int(float(y2) * new_height / h)
                        f.write(f"{label} {x1} {y1} {x2} {y2}\n")

print("Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.")

Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.


In [11]:
#BIRD ICIN NORMALİZE İŞLEMİ 1 W OPENCV

""" 
Açıklamalar:
normalize_image: Görselleri [0, 1] aralığında normalize eder.
update_label_coordinates: Etiketlerdeki koordinatları, normalleştirme işleminden sonra günceller.
Görselleri Normalleştirme: Görseller images klasöründen okunur, normalize edilir ve normalized_images adlı yeni bir klasöre kaydedilir.
Etiketleri Güncelleme: Görsellerin boyutu değişmediği için etiketlerin koordinatları aynı kalır, ancak bu süreçte dosya sistemi 
değişiklikleri gözlemlenebilir.
Bu kod, görselleri normalleştirir ve etiket dosyalarını günceller, ancak etiketlerin kendisini değiştirmez (örn. "bird" etiketi "bird" olarak kalır).
"""

' \nAçıklamalar:\nnormalize_image: Görselleri [0, 1] aralığında normalize eder.\nupdate_label_coordinates: Etiketlerdeki koordinatları, normalleştirme işleminden sonra günceller.\nGörselleri Normalleştirme: Görseller images klasöründen okunur, normalize edilir ve normalized_images adlı yeni bir klasöre kaydedilir.\nEtiketleri Güncelleme: Görsellerin boyutu değişmediği için etiketlerin koordinatları aynı kalır, ancak bu süreçte dosya sistemi \ndeğişiklikleri gözlemlenebilir.\nBu kod, görselleri normalleştirir ve etiket dosyalarını günceller, ancak etiketlerin kendisini değiştirmez (örn. "bird" etiketi "bird" olarak kalır).\n'

In [15]:
import os
import cv2
import numpy as np

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')

# Çıktı klasörleri
normalized_images_folder = os.path.join(dataset_folder, 'normalized_images')
normalized_labels_folder = os.path.join(dataset_folder, 'normalized_labels')
os.makedirs(normalized_images_folder, exist_ok=True)
os.makedirs(normalized_labels_folder, exist_ok=True)

def normalize_image(image):
    """Görseli [0, 1] aralığında normalize eder."""
    normalized_image = image.astype('float32') / 255.0
    return normalized_image

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * new_shape[1] / original_shape[1]
        new_y_center = y_center * new_shape[0] / original_shape[0]
        new_width = width * new_shape[1] / original_shape[1]
        new_height = height * new_shape[0] / original_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görselleri normalleştirme ve etiketleri güncelleme işlemi
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli yükle ve normalleştir
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)
        normalized_image = normalize_image(image)
        
        # Normalleştirilmiş görseli kaydet
        normalized_image_path = os.path.join(normalized_images_folder, file_name)
        cv2.imwrite(normalized_image_path, normalized_image * 255.0)  # Normalleştirilmiş görüntüyü [0, 255] aralığına geri döndür
        
        # Etiket dosyasını güncelle ve yeni klasöre kaydet
        if os.path.exists(label_path):
            updated_labels = update_label_coordinates(label_path, original_shape, original_shape)
            normalized_label_path = os.path.join(normalized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(normalized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Normalleştirme ve etiket güncelleme işlemi tamamlandı.")

Normalleştirme ve etiket güncelleme işlemi tamamlandı.


In [ ]:
#UAV ICIN NORMALİZE İŞLEMİ 1 W OPENCV

In [17]:
import os
import cv2
import numpy as np

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')

# Çıktı klasörleri
normalized_images_folder = os.path.join(dataset_folder, 'normalized_images')
normalized_labels_folder = os.path.join(dataset_folder, 'normalized_labels')
os.makedirs(normalized_images_folder, exist_ok=True)
os.makedirs(normalized_labels_folder, exist_ok=True)

def normalize_image(image):
    """Görseli [0, 1] aralığında normalize eder."""
    normalized_image = image.astype('float32') / 255.0
    return normalized_image

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * new_shape[1] / original_shape[1]
        new_y_center = y_center * new_shape[0] / original_shape[0]
        new_width = width * new_shape[1] / original_shape[1]
        new_height = height * new_shape[0] / original_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görselleri normalleştirme ve etiketleri güncelleme işlemi
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli yükle ve normalleştir
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)
        normalized_image = normalize_image(image)
        
        # Normalleştirilmiş görseli kaydet
        normalized_image_path = os.path.join(normalized_images_folder, file_name)
        cv2.imwrite(normalized_image_path, normalized_image * 255.0)  # Normalleştirilmiş görüntüyü [0, 255] aralığına geri döndür
        
        # Etiket dosyasını güncelle ve yeni klasöre kaydet
        if os.path.exists(label_path):
            updated_labels = update_label_coordinates(label_path, original_shape, original_shape)
            normalized_label_path = os.path.join(normalized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(normalized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Normalleştirme ve etiket güncelleme işlemi tamamlandı.")

Normalleştirme ve etiket güncelleme işlemi tamamlandı.


In [ ]:
#Bird_and_UAV ICIN NORMALİZE İŞLEMİ 1 W OPENCV

In [19]:
import os
import cv2
import numpy as np

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird_and_UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')

# Çıktı klasörleri
normalized_images_folder = os.path.join(dataset_folder, 'normalized_images')
normalized_labels_folder = os.path.join(dataset_folder, 'normalized_labels')
os.makedirs(normalized_images_folder, exist_ok=True)
os.makedirs(normalized_labels_folder, exist_ok=True)

def normalize_image(image):
    """Görseli [0, 1] aralığında normalize eder."""
    normalized_image = image.astype('float32') / 255.0
    return normalized_image

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * new_shape[1] / original_shape[1]
        new_y_center = y_center * new_shape[0] / original_shape[0]
        new_width = width * new_shape[1] / original_shape[1]
        new_height = height * new_shape[0] / original_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görselleri normalleştirme ve etiketleri güncelleme işlemi
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli yükle ve normalleştir
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)
        normalized_image = normalize_image(image)
        
        # Normalleştirilmiş görseli kaydet
        normalized_image_path = os.path.join(normalized_images_folder, file_name)
        cv2.imwrite(normalized_image_path, normalized_image * 255.0)  # Normalleştirilmiş görüntüyü [0, 255] aralığına geri döndür
        
        # Etiket dosyasını güncelle ve yeni klasöre kaydet
        if os.path.exists(label_path):
            updated_labels = update_label_coordinates(label_path, original_shape, original_shape)
            normalized_label_path = os.path.join(normalized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(normalized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Normalleştirme ve etiket güncelleme işlemi tamamlandı.")

Normalleştirme ve etiket güncelleme işlemi tamamlandı.


In [25]:
"""

AUGMENTED VERİLERİN LABEL İŞİNE BAKILACAK!!
1)artırılan verilerin isimleri değiştirilse iyi olur yoksa orjinalleriyle aynı olacaklar hem de 4 er tane
2) sonrasında tekrar labellama yapmak gerekebilir

"""

'\n\nAUGMENTED VERİLERİN LABEL İŞİNE BAKILACAK!!\n1)artırılan verilerin isimleri değiştirilse iyi olur yoksa orjinalleriyle aynı olacaklar hem de 4 er tane\n2) sonrasında tekrar labellama yapmak gerekebilir\n\n'

-------------------------------------------

OLMADI BURNDAN SONRAKİLERİ HEP DÜZELT LABELLARI GÜNCELLEYEMİYORUM

In [ ]:
#BIRD AND UAV İÇİN LABEL GÜNCELLEMELİ DATA AUGMENTATION

In [31]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import imgaug.augmenters as iaa
import shutil

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird_and_UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
augmented_images_folder = os.path.join(dataset_folder, 'augmented_images')
augmented_labels_folder = os.path.join(dataset_folder, 'augmented_labels')

# Çıktı klasörlerini oluşturun
os.makedirs(augmented_images_folder, exist_ok=True)
os.makedirs(augmented_labels_folder, exist_ok=True)

# Veri artırma işlemleri için imgaug kullanarak bir artırma zinciri tanımlayın
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Yatay çevirme
    iaa.Flipud(0.5),  # Dikey çevirme
    iaa.Affine(rotate=(-25, 25)),  # Rastgele döndürme
    iaa.Affine(scale=(0.8, 1.2)),  # Rastgele yakınlaştırma
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Rastgele gürültü ekleme
    iaa.Multiply((0.8, 1.2)),  # Renklendirme (görselin parlaklığını rastgele artırma/azaltma)
    iaa.LinearContrast((0.75, 1.5)),  # Kontrast artırma
    iaa.CropAndPad(percent=(0, 0.1)),  # Görseli rastgele kesme ve padleme
    iaa.GaussianBlur(sigma=(0, 3.0))  # Blurlama
])

def color_jitter(image):
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(np.random.uniform(0.8, 1.2))  # Rastgele renk ayarı
    return image

def update_label(label_path, augmenter, original_image_shape, augmented_image_shape):
    with open(label_path, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * augmented_image_shape[1] / original_image_shape[1]
        new_y_center = y_center * augmented_image_shape[0] / original_image_shape[0]
        new_width = width * augmented_image_shape[1] / original_image_shape[1]
        new_height = height * augmented_image_shape[0] / original_image_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")

    return updated_lines

# Etiket dosyalarını ve görselleri işle
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli aç
        image = Image.open(image_path)
        image_np = np.array(image)
        original_image_shape = image_np.shape

        # imgaug ile artırma
        image_aug_np = augmenter.augment_image(image_np)
        image_aug = Image.fromarray(image_aug_np)
        augmented_image_shape = image_aug_np.shape

        # Renklendirme işlemi
        image_aug = color_jitter(image_aug)

        # Artırılmış görüntüyü kaydet
        augmented_image_path = os.path.join(augmented_images_folder, file_name)
        image_aug.save(augmented_image_path)

        # Etiket dosyasını güncelle ve kaydet
        if os.path.exists(label_path):
            updated_labels = update_label(label_path, augmenter, original_image_shape, augmented_image_shape)
            augmented_label_path = os.path.join(augmented_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(augmented_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Veri artırma ve etiket güncelleme işlemi tamamlandı.")

Veri artırma ve etiket güncelleme işlemi tamamlandı.


In [ ]:
#BIRD İÇİN LABEL GÜNCELLEMELİ DATA AUGMENTATION

In [34]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import imgaug.augmenters as iaa
import shutil

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
augmented_images_folder = os.path.join(dataset_folder, 'augmented_images')
augmented_labels_folder = os.path.join(dataset_folder, 'augmented_labels')

# Çıktı klasörlerini oluşturun
os.makedirs(augmented_images_folder, exist_ok=True)
os.makedirs(augmented_labels_folder, exist_ok=True)

# Veri artırma işlemleri için imgaug kullanarak bir artırma zinciri tanımlayın
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Yatay çevirme
    iaa.Flipud(0.5),  # Dikey çevirme
    iaa.Affine(rotate=(-25, 25)),  # Rastgele döndürme
    iaa.Affine(scale=(0.8, 1.2)),  # Rastgele yakınlaştırma
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Rastgele gürültü ekleme
    iaa.Multiply((0.8, 1.2)),  # Renklendirme (görselin parlaklığını rastgele artırma/azaltma)
    iaa.LinearContrast((0.75, 1.5)),  # Kontrast artırma
    iaa.CropAndPad(percent=(0, 0.1)),  # Görseli rastgele kesme ve padleme
    iaa.GaussianBlur(sigma=(0, 3.0))  # Blurlama
])

def color_jitter(image):
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(np.random.uniform(0.8, 1.2))  # Rastgele renk ayarı
    return image

def update_label(label_path, augmenter, original_image_shape, augmented_image_shape):
    with open(label_path, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * augmented_image_shape[1] / original_image_shape[1]
        new_y_center = y_center * augmented_image_shape[0] / original_image_shape[0]
        new_width = width * augmented_image_shape[1] / original_image_shape[1]
        new_height = height * augmented_image_shape[0] / original_image_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")

    return updated_lines

# Etiket dosyalarını ve görselleri işle
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli aç
        image = Image.open(image_path)
        image_np = np.array(image)
        original_image_shape = image_np.shape

        # imgaug ile artırma
        image_aug_np = augmenter.augment_image(image_np)
        image_aug = Image.fromarray(image_aug_np)
        augmented_image_shape = image_aug_np.shape

        # Renklendirme işlemi
        image_aug = color_jitter(image_aug)

        # Artırılmış görüntüyü kaydet
        augmented_image_path = os.path.join(augmented_images_folder, file_name)
        image_aug.save(augmented_image_path)

        # Etiket dosyasını güncelle ve kaydet
        if os.path.exists(label_path):
            updated_labels = update_label(label_path, augmenter, original_image_shape, augmented_image_shape)
            augmented_label_path = os.path.join(augmented_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(augmented_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Veri artırma ve etiket güncelleme işlemi tamamlandı.")

Veri artırma ve etiket güncelleme işlemi tamamlandı.


In [ ]:
#UAV İÇİN LABEL GÜNCELLEMELİ DATA AUGMENTATION

In [36]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import imgaug.augmenters as iaa
import shutil

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
augmented_images_folder = os.path.join(dataset_folder, 'augmented_images')
augmented_labels_folder = os.path.join(dataset_folder, 'augmented_labels')

# Çıktı klasörlerini oluşturun
os.makedirs(augmented_images_folder, exist_ok=True)
os.makedirs(augmented_labels_folder, exist_ok=True)

# Veri artırma işlemleri için imgaug kullanarak bir artırma zinciri tanımlayın
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Yatay çevirme
    iaa.Flipud(0.5),  # Dikey çevirme
    iaa.Affine(rotate=(-25, 25)),  # Rastgele döndürme
    iaa.Affine(scale=(0.8, 1.2)),  # Rastgele yakınlaştırma
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Rastgele gürültü ekleme
    iaa.Multiply((0.8, 1.2)),  # Renklendirme (görselin parlaklığını rastgele artırma/azaltma)
    iaa.LinearContrast((0.75, 1.5)),  # Kontrast artırma
    iaa.CropAndPad(percent=(0, 0.1)),  # Görseli rastgele kesme ve padleme
    iaa.GaussianBlur(sigma=(0, 3.0))  # Blurlama
])

def color_jitter(image):
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(np.random.uniform(0.8, 1.2))  # Rastgele renk ayarı
    return image

def update_label(label_path, augmenter, original_image_shape, augmented_image_shape):
    with open(label_path, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * augmented_image_shape[1] / original_image_shape[1]
        new_y_center = y_center * augmented_image_shape[0] / original_image_shape[0]
        new_width = width * augmented_image_shape[1] / original_image_shape[1]
        new_height = height * augmented_image_shape[0] / original_image_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")

    return updated_lines

# Etiket dosyalarını ve görselleri işle
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli aç
        image = Image.open(image_path)
        image_np = np.array(image)
        original_image_shape = image_np.shape

        # imgaug ile artırma
        image_aug_np = augmenter.augment_image(image_np)
        image_aug = Image.fromarray(image_aug_np)
        augmented_image_shape = image_aug_np.shape

        # Renklendirme işlemi
        image_aug = color_jitter(image_aug)

        # Artırılmış görüntüyü kaydet
        augmented_image_path = os.path.join(augmented_images_folder, file_name)
        image_aug.save(augmented_image_path)

        # Etiket dosyasını güncelle ve kaydet
        if os.path.exists(label_path):
            updated_labels = update_label(label_path, augmenter, original_image_shape, augmented_image_shape)
            augmented_label_path = os.path.join(augmented_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(augmented_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Veri artırma ve etiket güncelleme işlemi tamamlandı.")

Veri artırma ve etiket güncelleme işlemi tamamlandı.


In [44]:
#BIRD AND UAV İÇİN LABEL GÜNCELLEMELİ NORMALİZE İŞLEMİ
"""
Açıklamalar:
Normalizasyon: Görseli [0, 1] aralığına normalize etmek için normalize_image fonksiyonu kullanılır. Bu işlemden sonra, görseli tekrar [0, 255] aralığına döndürüp kaydediyoruz.

Koordinat Güncelleme: update_label_coordinates fonksiyonu, etiketlerdeki koordinatları yeni boyutlara göre günceller. Bu işlemi görselin orijinal boyutları ve normalleştirilmiş boyutları arasındaki farkı hesaplayarak yapıyoruz.

Görsel ve Etiket Güncelleme: Her görsel için etiketleri de güncelleyerek yeni bir dosya olarak kaydediyoruz.
"""

'\nAçıklamalar:\nNormalizasyon: Görseli [0, 1] aralığına normalize etmek için normalize_image fonksiyonu kullanılır. Bu işlemden sonra, görseli tekrar [0, 255] aralığına döndürüp kaydediyoruz.\n\nKoordinat Güncelleme: update_label_coordinates fonksiyonu, etiketlerdeki koordinatları yeni boyutlara göre günceller. Bu işlemi görselin orijinal boyutları ve normalleştirilmiş boyutları arasındaki farkı hesaplayarak yapıyoruz.\n\nGörsel ve Etiket Güncelleme: Her görsel için etiketleri de güncelleyerek yeni bir dosya olarak kaydediyoruz.\n'

In [38]:
import os
import cv2
import numpy as np

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird_and_UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')

# Çıktı klasörleri
normalized_images_folder = os.path.join(dataset_folder, 'normalized_images')
normalized_labels_folder = os.path.join(dataset_folder, 'normalized_labels')
os.makedirs(normalized_images_folder, exist_ok=True)
os.makedirs(normalized_labels_folder, exist_ok=True)

def normalize_image(image):
    """Görseli [0, 1] aralığında normalize eder."""
    normalized_image = image.astype('float32') / 255.0
    return normalized_image

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * new_shape[1] / original_shape[1]
        new_y_center = y_center * new_shape[0] / original_shape[0]
        new_width = width * new_shape[1] / original_shape[1]
        new_height = height * new_shape[0] / original_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görselleri normalleştirme ve etiketleri güncelleme işlemi
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli yükle ve normalleştir
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)
        normalized_image = normalize_image(image)
        
        # Normalleştirilmiş görseli kaydet
        normalized_image_path = os.path.join(normalized_images_folder, file_name)
        cv2.imwrite(normalized_image_path, normalized_image * 255.0)  # Normalleştirilmiş görüntüyü [0, 255] aralığına geri döndür
        
        # Etiket dosyasını güncelle ve yeni klasöre kaydet
        if os.path.exists(label_path):
            # Burada new_shape, normalize edilmiş görüntünün boyutu olacaktır, ki bu orijinal görüntü ile aynı olmalı
            new_shape = normalized_image.shape[:2]
            updated_labels = update_label_coordinates(label_path, original_shape, new_shape)
            normalized_label_path = os.path.join(normalized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(normalized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Normalleştirme ve etiket güncelleme işlemi tamamlandı.")

Normalleştirme ve etiket güncelleme işlemi tamamlandı.


In [ ]:
#BIRD İÇİN LABEL GÜNCELLEMELİ NORMALİZE İŞLEMİ

In [40]:
import os
import cv2
import numpy as np

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')

# Çıktı klasörleri
normalized_images_folder = os.path.join(dataset_folder, 'normalized_images')
normalized_labels_folder = os.path.join(dataset_folder, 'normalized_labels')
os.makedirs(normalized_images_folder, exist_ok=True)
os.makedirs(normalized_labels_folder, exist_ok=True)

def normalize_image(image):
    """Görseli [0, 1] aralığında normalize eder."""
    normalized_image = image.astype('float32') / 255.0
    return normalized_image

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * new_shape[1] / original_shape[1]
        new_y_center = y_center * new_shape[0] / original_shape[0]
        new_width = width * new_shape[1] / original_shape[1]
        new_height = height * new_shape[0] / original_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görselleri normalleştirme ve etiketleri güncelleme işlemi
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli yükle ve normalleştir
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)
        normalized_image = normalize_image(image)
        
        # Normalleştirilmiş görseli kaydet
        normalized_image_path = os.path.join(normalized_images_folder, file_name)
        cv2.imwrite(normalized_image_path, normalized_image * 255.0)  # Normalleştirilmiş görüntüyü [0, 255] aralığına geri döndür
        
        # Etiket dosyasını güncelle ve yeni klasöre kaydet
        if os.path.exists(label_path):
            # Burada new_shape, normalize edilmiş görüntünün boyutu olacaktır, ki bu orijinal görüntü ile aynı olmalı
            new_shape = normalized_image.shape[:2]
            updated_labels = update_label_coordinates(label_path, original_shape, new_shape)
            normalized_label_path = os.path.join(normalized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(normalized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Normalleştirme ve etiket güncelleme işlemi tamamlandı.")

Normalleştirme ve etiket güncelleme işlemi tamamlandı.


In [ ]:
#UAV İÇİN LABEL GÜNCELLEMELİ NORMALİZE İŞLEMİ

In [42]:
import os
import cv2
import numpy as np

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')

# Çıktı klasörleri
normalized_images_folder = os.path.join(dataset_folder, 'normalized_images')
normalized_labels_folder = os.path.join(dataset_folder, 'normalized_labels')
os.makedirs(normalized_images_folder, exist_ok=True)
os.makedirs(normalized_labels_folder, exist_ok=True)

def normalize_image(image):
    """Görseli [0, 1] aralığında normalize eder."""
    normalized_image = image.astype('float32') / 255.0
    return normalized_image

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Koordinatları yeni boyutlara göre güncelle
        new_x_center = x_center * new_shape[1] / original_shape[1]
        new_y_center = y_center * new_shape[0] / original_shape[0]
        new_width = width * new_shape[1] / original_shape[1]
        new_height = height * new_shape[0] / original_shape[0]
        
        updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görselleri normalleştirme ve etiketleri güncelleme işlemi
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        
        # Görseli yükle ve normalleştir
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)
        normalized_image = normalize_image(image)
        
        # Normalleştirilmiş görseli kaydet
        normalized_image_path = os.path.join(normalized_images_folder, file_name)
        cv2.imwrite(normalized_image_path, normalized_image * 255.0)  # Normalleştirilmiş görüntüyü [0, 255] aralığına geri döndür
        
        # Etiket dosyasını güncelle ve yeni klasöre kaydet
        if os.path.exists(label_path):
            # Burada new_shape, normalize edilmiş görüntünün boyutu olacaktır, ki bu orijinal görüntü ile aynı olmalı
            new_shape = normalized_image.shape[:2]
            updated_labels = update_label_coordinates(label_path, original_shape, new_shape)
            normalized_label_path = os.path.join(normalized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(normalized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Normalleştirme ve etiket güncelleme işlemi tamamlandı.")

Normalleştirme ve etiket güncelleme işlemi tamamlandı.


In [46]:
#BIRD AND UAV İÇİN LABEL GÜNCELLEMELİ RESIZE İŞLEMİ
"""
Açıklamalar:
Yeniden Boyutlandırma: Görüntüler cv2.resize kullanılarak yeniden boyutlandırılır.

Etiket Güncelleme: update_label_coordinates fonksiyonu, etiketlerdeki merkez koordinatları ve boyutları yeni boyutlara göre günceller. YOLO formatı kullanıldığından, koordinatlar ve boyutlar normalizasyon yapılmış şekilde yeniden hesaplanır.

Dosya Yolu Güncellemeleri: Etiket dosyalarını güncellemek için eski etiket dosyalarını okuyup güncellenmiş etiketlerle yeni dosyalar oluşturulur.
"""

'\nAçıklamalar:\nYeniden Boyutlandırma: Görüntüler cv2.resize kullanılarak yeniden boyutlandırılır.\n\nEtiket Güncelleme: update_label_coordinates fonksiyonu, etiketlerdeki merkez koordinatları ve boyutları yeni boyutlara göre günceller. YOLO formatı kullanıldığından, koordinatlar ve boyutlar normalizasyon yapılmış şekilde yeniden hesaplanır.\n\nDosya Yolu Güncellemeleri: Etiket dosyalarını güncellemek için eski etiket dosyalarını okuyup güncellenmiş etiketlerle yeni dosyalar oluşturulur.\n'

In [48]:
import os
import cv2

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird_and_UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
resized_images_folder = os.path.join(dataset_folder, 'resized_images')
resized_labels_folder = os.path.join(dataset_folder, 'resized_labels')

# Yeniden boyutlandırma hedef boyutları
new_width = 256
new_height = 256

# Çıktı klasörlerini oluşturun
os.makedirs(resized_images_folder, exist_ok=True)
os.makedirs(resized_labels_folder, exist_ok=True)

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id = parts[0]
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            
            # Koordinatları yeni boyutlara göre güncelle
            new_x_center = x_center * new_shape[1] / original_shape[1]
            new_y_center = y_center * new_shape[0] / original_shape[0]
            new_width = width * new_shape[1] / original_shape[1]
            new_height = height * new_shape[0] / original_shape[0]
            
            updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görüntüleri ve etiketleri işleyin
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        # Görüntü dosyasını oku
        image_path = os.path.join(images_folder, file_name)
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)

        # Görüntüyü yeniden boyutlandır
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        resized_image_path = os.path.join(resized_images_folder, file_name)
        cv2.imwrite(resized_image_path, resized_image)

        # Etiket dosyasını oku ve güncelle
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        if os.path.exists(label_path):
            updated_labels = update_label_coordinates(label_path, original_shape, (new_height, new_width))
            resized_label_path = os.path.join(resized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(resized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.")

Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.


In [ ]:
#BIRD ICIN LABEL GÜNCELLEMELİ RESIZE İŞLEMİ

In [50]:
import os
import cv2

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/Bird'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
resized_images_folder = os.path.join(dataset_folder, 'resized_images')
resized_labels_folder = os.path.join(dataset_folder, 'resized_labels')

# Yeniden boyutlandırma hedef boyutları
new_width = 256
new_height = 256

# Çıktı klasörlerini oluşturun
os.makedirs(resized_images_folder, exist_ok=True)
os.makedirs(resized_labels_folder, exist_ok=True)

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id = parts[0]
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            
            # Koordinatları yeni boyutlara göre güncelle
            new_x_center = x_center * new_shape[1] / original_shape[1]
            new_y_center = y_center * new_shape[0] / original_shape[0]
            new_width = width * new_shape[1] / original_shape[1]
            new_height = height * new_shape[0] / original_shape[0]
            
            updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görüntüleri ve etiketleri işleyin
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        # Görüntü dosyasını oku
        image_path = os.path.join(images_folder, file_name)
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)

        # Görüntüyü yeniden boyutlandır
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        resized_image_path = os.path.join(resized_images_folder, file_name)
        cv2.imwrite(resized_image_path, resized_image)

        # Etiket dosyasını oku ve güncelle
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        if os.path.exists(label_path):
            updated_labels = update_label_coordinates(label_path, original_shape, (new_height, new_width))
            resized_label_path = os.path.join(resized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(resized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.")


Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.


In [ ]:
#UAV İÇİN LABEL GÜNCELLEMELİ RESIZE İŞLEMİ

In [52]:
import os
import cv2

# Klasör yolları
dataset_folder = 'C:/Users/edana/Desktop/dataset_new/UAV'
images_folder = os.path.join(dataset_folder, 'images')
labels_folder = os.path.join(dataset_folder, 'labels')
resized_images_folder = os.path.join(dataset_folder, 'resized_images')
resized_labels_folder = os.path.join(dataset_folder, 'resized_labels')

# Yeniden boyutlandırma hedef boyutları
new_width = 256
new_height = 256

# Çıktı klasörlerini oluşturun
os.makedirs(resized_images_folder, exist_ok=True)
os.makedirs(resized_labels_folder, exist_ok=True)

def update_label_coordinates(label_path, original_shape, new_shape):
    """Etiket dosyasındaki koordinatları yeniden boyutlandırmaya göre günceller."""
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id = parts[0]
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            
            # Koordinatları yeni boyutlara göre güncelle
            new_x_center = x_center * new_shape[1] / original_shape[1]
            new_y_center = y_center * new_shape[0] / original_shape[0]
            new_width = width * new_shape[1] / original_shape[1]
            new_height = height * new_shape[0] / original_shape[0]
            
            updated_lines.append(f"{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n")
    
    return updated_lines

# Görüntüleri ve etiketleri işleyin
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.webp')):
        # Görüntü dosyasını oku
        image_path = os.path.join(images_folder, file_name)
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]  # (yükseklik, genişlik)

        # Görüntüyü yeniden boyutlandır
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        resized_image_path = os.path.join(resized_images_folder, file_name)
        cv2.imwrite(resized_image_path, resized_image)

        # Etiket dosyasını oku ve güncelle
        label_path = os.path.join(labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
        if os.path.exists(label_path):
            updated_labels = update_label_coordinates(label_path, original_shape, (new_height, new_width))
            resized_label_path = os.path.join(resized_labels_folder, file_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt').replace('.webp', '.txt'))
            with open(resized_label_path, 'w') as label_file:
                label_file.writelines(updated_labels)

print("Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.")


Görüntülerin yeniden boyutlandırılması ve etiketlerin güncellenmesi tamamlandı.
